In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd

import config
import sys
print(sys.version)

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


In [2]:
assert os.path.exists(config.conceptnet_path)
conceptnet_df = pd.read_csv(config.conceptnet_path, names=["uri", "relation", "snode", "enode", "more"], sep="\t")

In [4]:
START_ID = '<START_ID>'
END_ID = '<END_ID>'
PAD_ID = '<PAD_ID>'
UNK_ID = '<UNK_ID>'

import tensorlayer as tl

vocab = tl.nlp.Vocabulary(
    config.news20_vocab_path, start_word=START_ID, end_word=END_ID, unk_word=UNK_ID
)

INFO:tensorflow:Initializing vocabulary from file: ../data/20-newsgroups/clean/vocab.txt


[TL] Vocabulary from ../data/20-newsgroups/clean/vocab.txt : <START_ID> <END_ID> <UNK_ID>


[TL]     vocabulary with 23568 words (includes start_word, end_word, unk_word)


[TL]       start_id: 35


[TL]       end_id  : 34


In [237]:
# print(df.loc[0])
# ndf = conceptnet_df[conceptnet_df["snode"] == "/c/en/%s" % "atheism"]
# print(ndf[["relation", "snode", "enode"]])
# ndf = conceptnet_df[conceptnet_df["enode"] == "/c/en/%s" % "atheism"]
# print(ndf[["relation", "snode", "enode"]])
# info = eval(conceptnet_df["more"].loc[0])
# print(info)
# print(info["weight"])

In [5]:
def preprocess(text):
    ntext = text.split("/")[3] if "/" in text else text
    ntext = re.sub(r'[\W_]+', ' ', ntext)
    textlist = ntext.split()
    textlist = [t for t in textlist if vocab.word_to_id(t) != vocab.unk_id and len(t) >= 3]
    
    if len(textlist) <= 1:
        return textlist
    else:
        text_with_tag = nltk.pos_tag(textlist)  # a list of words a list of words with part of speech
        selected = [text_tag[0] for text_tag in text_with_tag \
                    if text_tag[1] in config.pos_dict and \
                    (config.pos_dict[text_tag[1]] == 'a' or config.pos_dict[text_tag[1]] == 'n')]
        return selected

print(preprocess("/c/en/learning_about_science/n"))
print(preprocess("/c/en/atheism/n"))
print(preprocess("/c/en/atheism_a/n"))
print(preprocess("atheism"))
print(preprocess("/c/en/christan_science"))
print(preprocess("dkjflasdk"))

['science']
['atheism']
['atheism']
['atheism']
['science']
[]


In [212]:
glove_dict = dict()
glove_filename = config.word_embed_file_path
with open(glove_filename, 'r') as f:
    for idx, line in enumerate(f):
        content = line.replace("\n", "").split(" ")

        word = content[0]
        vect = np.array(content[1:]).astype(np.float32)
        
        glove_dict[word] = vect

from scipy.spatial.distance import cosine

print(cosine(glove_dict["religion"], glove_dict["atheism"]))
print(cosine(glove_dict["religion"], glove_dict["science"]))
print(cosine(glove_dict["atheism"], glove_dict["science"]))
print(cosine(glove_dict["mac"], glove_dict["windows"]))

0.523052453994751
0.5494254529476166
0.8710274249315262
0.6473370492458344


In [6]:
text_df = pd.read_csv(config.news20_train_path)

sample_text = preprocess(text_df["selected_tfidf"].loc[0])
print(sample_text[:])
print(vocab.word_to_id("technology"))

['free', 'energy', 'technology', 'free', 'energy', 'technology', 'robert', 'mcelwaine', 'physicist', 'heavier', 'air', 'machine', 'laws', 'example', 'simon', 'demonstration', 'possible', 'known', 'substances', 'forms', 'machinery', 'forms', 'machine', 'long', 'distances', 'air', 'complete', 'possible', 'demonstration', 'physical', 'fact', 'such', 'wright', 'such', 'pronouncements', 'take', 'airplanes', 'physicists', 'other', 'things', 'several', 'free', 'energy', 'terms', 'such', 'pseudo', 'science', 'perpetual', 'motion', 'laws', 'energy', 'law', 'thermodynamics', 'energy', 'law', 'physicists', 'certain', 'things', 'things', 'such', 'common', 'orthodox', 'modern', 'science', 'cover', 'inconsistencies', 'contradictions', 'modern', 'theories', 'free', 'energy', 'devices', 'unlimited', 'supply', 'energy', 'universe', 'fuel', 'world', 'wide', 'energy', 'crisis', 'pollution', 'degradation', 'depletion', 'environment', 'free', 'energy', 'devices', 'energy', 'tap', 'energy', 'sources', 'form

In [103]:
related_relation_set = set([
    '/r/RelatedTo'
])


positive_relation_set = set([
    '/r/IsA',  '/r/PartOf',  
    '/r/AtLocation', 
    '/r/Causes', '/r/FormOf'
    '/r/Synonym', '/r/CreatedBy',
    '/r/HasProperty', '/r/UsedFor',
    '/r/HasA', '/r/HasContext'
])

negative_relation_set = set([
    '/r/Antonym', '/r/DistinctFrom',
    '/r/NotIsA', '/r/NotHasProperty',
    '/r/NotUsedFor'
])

print("/r/IsA" in positive_relation_set)
print("/r/NotIsA" in positive_relation_set)
print("/r/IsA" in negative_relation_set)
print("/r/NotIsA" in negative_relation_set)
print(type(positive_relation_set))
print(type(negative_relation_set))

True
False
False
True
<class 'set'>
<class 'set'>


In [238]:
def remove_word(word):
    pword = preprocess(word)
    pword = ' '.join(pword)
    if vocab.word_to_id(pword) == vocab.unk_id or len(pword) < 3:
        return np.nan
    else:
        return pword

def trinary_relation(relation):
    if relation in positive_relation_set:
        return "positive"
    elif relation in negative_relation_set:
        return "negative"
    elif relation in related_relation_set:
        return "related"
    else:
        return np.nan

def preprocess_conceptnet(df):
    ndf = df[["snode", "enode", "relation", "more"]]
    ndf["relation"] = ndf["relation"].apply(lambda r: trinary_relation(r))
    ndf = ndf.dropna()
    ndf = ndf.drop_duplicates()
    ndf["weight"] = ndf["more"].apply(lambda w: eval(w)["weight"])
    ndf["snode"] = ndf["snode"].apply(lambda w: remove_word(w))
    ndf["enode"] = ndf["enode"].apply(lambda w: remove_word(w))
    ndf = ndf.dropna()
    ndf = ndf.drop_duplicates()
    return ndf

# print(remove_word("/c/en/learning_about_science"))

In [ ]:
processed_conceptnet_df = preprocess_conceptnet(conceptnet_df)

In [ ]:
processed_conceptnet_df = processed_conceptnet_df[["snode", "enode", "relation", "weight"]]

In [284]:
processed_conceptnet_df = processed_conceptnet_df[processed_conceptnet_df["weight"] >= 0.9999]

In [294]:
# print(vocab.word_to_id("unbelief"))
# print(vocab.unk_id)
# print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "atheism"][processed_conceptnet_df["enode"] == "theism"])
# print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "atheism"][processed_conceptnet_df["snode"] == "theism"])
# print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "atheism"][processed_conceptnet_df["enode"] == "religion"])
# print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "atheism"][processed_conceptnet_df["snode"] == "religion"])
# print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "atheism"][processed_conceptnet_df["enode"] == "science"])
# print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "atheism"][processed_conceptnet_df["snode"] == "science"])
print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "atheism"])
print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "atheism"])
print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "science"])
print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "science"])
# print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "belief"][processed_conceptnet_df["enode"] == "religion"])
# print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "belief"][processed_conceptnet_df["snode"] == "religion"])
# print(processed_conceptnet_df[processed_conceptnet_df["snode"] == "religion"][processed_conceptnet_df["enode"] == "science"])
# print(processed_conceptnet_df[processed_conceptnet_df["enode"] == "religion"][processed_conceptnet_df["snode"] == "science"])

           snode        enode  relation  weight
679666   atheism   historical  positive     1.0
951697   atheism       belief  positive     1.0
1301118  atheism  agnosticism   related     1.0
1301126  atheism    pantheism   related     1.0
1301129  atheism       theism   related     1.0
1301130  atheism      atheist   related     2.0
1301131  atheism    atheistic   related     1.0
             snode    enode  relation  weight
77613      science  atheism  positive     1.0
1301137    atheist  atheism   related     1.0
1301142    atheist  atheism   related     2.0
1301157  atheistic  atheism   related     1.0
1799306        god  atheism   related     2.0
2623916     theism  atheism   related     1.0
           snode          enode  relation  weight
40051    science         school  positive   1.000
40053    science     technology  positive   1.000
40054    science     university  positive   5.657
77611    science      accidents  positive   1.000
77613    science        atheism  positive   

In [286]:
def simplify_df(df):
    relation_dict = dict()
    for idx, row in df.iterrows():
        snode = row["snode"]
        enode = row["enode"]
        if snode == enode:
            continue
        relation = row["relation"]
        
        if snode not in relation_dict:
            relation_dict[snode] = {"positive": set(), "negative": set(), "related": set()}
        if enode not in relation_dict:
            relation_dict[enode] = {"positive": set(), "negative": set(), "related": set()}
            
        if relation == "related":
            if enode not in relation_dict[snode]["positive"] and enode not in relation_dict[snode]["negative"]:
                relation_dict[snode][relation].add(enode)
            if snode not in relation_dict[enode]["positive"] and snode not in relation_dict[enode]["negative"]:
                relation_dict[enode][relation].add(snode)
        elif relation == "positive" or relation == "negative":
            relation_dict[snode][relation].add(enode)
            relation_dict[enode][relation].add(snode)
            if enode in relation_dict[snode]["related"]:
                relation_dict[snode]["related"].remove(enode)
            if snode in relation_dict[enode]["related"]:
                relation_dict[enode]["related"].remove(snode)
        else:
            raise Exception("Invalid relation: %s" % relation)
    return relation_dict
        

simplified_conceptnet_dict = simplify_df(processed_conceptnet_df)

In [288]:
print(simplified_conceptnet_dict["atheism"]["positive"])
print(simplified_conceptnet_dict["atheism"]["negative"])
print(simplified_conceptnet_dict["atheism"]["related"])

print(simplified_conceptnet_dict["religion"]["positive"])
print(simplified_conceptnet_dict["religion"]["negative"])
print(simplified_conceptnet_dict["religion"]["related"])

{'science', 'belief', 'historical'}
set()
{'pantheism', 'atheistic', 'agnosticism', 'atheist', 'god', 'theism'}
{'sunday', 'tongue', 'propagation', 'hinduism', 'affirmation', 'damnation', 'reader', 'stem', 'immortality', 'christian', 'masses', 'gideon', 'church', 'judaism', 'buddhism', 'halo', 'hands', 'consecration', 'life', 'deliver', 'jehovah', 'pastoral', 'feet', 'paradise', 'psalm', 'falsehood', 'confirm', 'goddess', 'toleration', 'meditation', 'islamic', 'creator', 'chastity', 'cult', 'heavens', 'taoism', 'noah', 'revert', 'elijah', 'fundamentalism', 'devil', 'god', 'adversary', 'episcopal', 'rule', 'icon', 'jah', 'protestant', 'worship', 'lord', 'language', 'pray', 'weak', 'book', 'theism', 'satan', 'jumbo', 'anglican', 'office', 'use', 'persecution', 'antichrist', 'tablet', 'confess', 'brother', 'schism', 'unknowable', 'total', 'paganism', 'felix', 'canonical', 'crown', 'wars', 'clean', 'point', 'the', 'building', 'salvation', 'mormon', 'mysticism', 'reincarnate', 'habit', 'rat

In [326]:
def BFS(nodelist, relation_df, max_hop):
    positive_node_dict = {"pending": dict(), "complete": dict()}
    negative_node_dict = {"pending": dict(), "complete": dict()}
    # neutral_node_dict = {"pending": set(), "complete": set()}
    related_node_dict = {"pending": dict(), "complete": dict()}
    controversial_node_dict = {"pending": dict(), "complete": dict()}
    
    hop_dict = dict()
    
    def path_positive(word, hop):
        if word in related_node_dict["pending"]:
            related_node_dict["pending"].pop(word)
            positive_node_dict["pending"][word] = hop
        elif word in related_node_dict["complete"]:
            related_node_dict["complete"].pop(word)
            positive_node_dict["pending"][word] = hop
        elif word in negative_node_dict["pending"]:
            if negative_node_dict["pending"][word] == hop:
                negative_node_dict["pending"].pop(word)
                controversial_node_dict["complete"][word] = hop
            elif negative_node_dict["pending"][word] > hop:
                negative_node_dict["pending"].pop(word)
                positive_node_dict["pending"][word] = hop
            else:
                pass # do nothing
        elif word in negative_node_dict["complete"]:
            if negative_node_dict["complete"][word] == hop:
                negative_node_dict["complete"].pop(word)
                controversial_node_dict["complete"][word] = hop
            elif negative_node_dict["complete"][word] > hop:
                negative_node_dict["complete"].pop(word)
                positive_node_dict["pending"][word] = hop
            else:
                pass # do nothing
        elif word in positive_node_dict["pending"] or word in positive_node_dict["complete"]:
            pass # do nothing
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else: # from new
            positive_node_dict["pending"][word] = hop
    
    def path_negative(word, hop):
        if word in related_node_dict["pending"]:
            related_node_dict["pending"].pop(word)
            negative_node_dict["pending"][word] = hop
        elif word in related_node_dict["complete"]:
            related_node_dict["complete"].pop(word)
            negative_node_dict["pending"][word] = hop
        elif word in negative_node_dict["pending"] or word in negative_node_dict["complete"]:
            pass # do nothing
        elif word in positive_node_dict["pending"]:
            if positive_node_dict["pending"][word] == hop:
                positive_node_dict["pending"].pop(word)
                controversial_node_dict["complete"][word] = hop
            elif positive_node_dict["pending"][word] > hop:
                positive_node_dict["pending"].pop(word)
                negative_node_dict["pending"][word] = hop
            else:
                pass # do nothing
        elif word in positive_node_dict["complete"]:
            if positive_node_dict["complete"][word] == hop:
                positive_node_dict["complete"].pop(word)
                controversial_node_dict["complete"][word] = hop
            elif positive_node_dict["complete"][word] > hop:
                positive_node_dict["complete"].pop(word)
                negative_node_dict["pending"][word] = hop
            else:
                pass # do nothing
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else:
            negative_node_dict["pending"][word] = hop
    
    def path_related(word, hop):
        if word in related_node_dict["pending"] or word in related_node_dict["complete"]:
            pass # do nothing
        elif word in negative_node_dict["pending"] or word in negative_node_dict["complete"]:
            pass # do nothing
        elif word in positive_node_dict["pending"] or word in positive_node_dict["complete"]:
            pass # do nothing
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else:
            related_node_dict["pending"][word] = hop
    '''
    def path_controversial(word, hop):
        if word in related_node_dict["pending"]:
            related_node_dict["pending"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in related_node_dict["complete"]:
            related_node_dict["complete"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in negative_node_dict["pending"]:
            negative_node_dict["pending"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in negative_node_dict["complete"]:
            negative_node_dict["complete"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in positive_node_dict["pending"]:
            positive_node_dict["pending"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in positive_node_dict["complete"]:
            positive_node_dict["complete"].remove(word)
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
        elif word in controversial_node_dict["pending"] or word in controversial_node_dict["complete"]:
            pass # do nothing
        else: # from new
            # controversial_node_dict["pending"].add(word)
            controversial_node_dict["complete"].add(word)
    '''

    for node in nodelist:
        positive_node_dict["pending"][node] = 0
        hop_dict[node] = 0
    
    print("BFS ", " ".join(nodelist))
    round = 0
    while True:
        
        round += 1
        if round % 1000 == 0:
            print(
                round,
                # len(neutral_node_dict["pending"]),
                len(controversial_node_dict["pending"]),
                len(positive_node_dict["pending"]), 
                len(negative_node_dict["pending"]),
                len(related_node_dict["pending"])
            )

        # do nothing search neutral words anymore
        # assert len(neutral_node_dict["pending"]) == 0

        if len(controversial_node_dict["pending"]) > 0:
        #     current_node, saved_hop = controversial_node_dict["pending"].popitem()
        #     current_type = "controversial"
        #     controversial_node_dict["complete"][current_node] = saved_hop
            # print(current_node, controversial_node_dict["pending"])
            # print(positive_node_dict["pending"], positive_node_dict["complete"])
            # print(negative_node_dict["pending"], negative_node_dict["complete"])
            # print(related_node_dict["pending"], related_node_dict["complete"])
            # return
            raise Exception("Pending controversial")
        # elif len(neutral_node_dict["pending"]) > 0:
        #     current_node = neutral_node_dict["pending"].pop()
        #     current_type = "neutral"
        #     neutral_node_dict["complete"].add(current_node)
        #     raise Exception("Neutral words should not be searched")
        elif len(positive_node_dict["pending"]) > 0:
            current_node, saved_hop = positive_node_dict["pending"].popitem() 
            current_type = "positive"
            positive_node_dict["complete"][current_node] = saved_hop
        elif len(negative_node_dict["pending"]) > 0:
            current_node, saved_hop = negative_node_dict["pending"].popitem()
            current_type = "negative"
            negative_node_dict["complete"][current_node] = saved_hop
        elif len(related_node_dict["pending"]) > 0:
            current_node, saved_hop = related_node_dict["pending"].popitem()
            current_type = "related"
            related_node_dict["complete"][current_node] = saved_hop
        else:
            print("End")
            break

        current_hop = hop_dict[current_node] + 1
        if current_hop > max_hop:
            continue
        
        df1 = relation_df[relation_df["snode"] == current_node]
        df2 = relation_df[relation_df["enode"] == current_node]
        current_df = pd.concat([df1, df2])
        
        current_positive_relation_df = current_df[current_df["relation"] == "positive"]
        current_negative_relation_df = current_df[current_df["relation"] == "negative"]
        current_related_relation_df = current_df[current_df["relation"] == "related"]
        
        for idx, row in current_positive_relation_df.iterrows():
            node = row["snode"] if row["snode"] != current_node else row["enode"]
            if node == current_node:
                continue
            assert node != current_node
            
            if current_type == "positive": # current node is from positive_set
                path_positive(node, current_hop)
            elif current_type == "negative": # current node is from negative_set
                path_negative(node, current_hop)
            elif current_type == "related":
                path_related(node, current_hop)
            # elif current_type == "controversial":
            #     path_controversial(node)
            else:
                raise Exception("Type error")
                
            if node not in hop_dict or hop_dict[node] > current_hop:
                hop_dict[node] = current_hop
            
        for idx, row in current_negative_relation_df.iterrows():
            node = row["snode"] if row["snode"] != current_node else row["enode"]
            if node == current_node:
                continue
            assert node != current_node
            
            if current_type == "positive": # current node is from positive_set
                path_negative(node, current_hop)
            elif current_type == "negative": # current node is from negative_set
                path_related(node, current_hop)
            elif current_type == "related":
                path_related(node, current_hop)
            # elif current_type == "controversial":
            #     path_controversial(node)
            else:
                raise Exception("Type error")

            if node not in hop_dict or hop_dict[node] > current_hop:
                hop_dict[node] = current_hop
        
        for idx, row in current_related_relation_df.iterrows():
            node = row["snode"] if row["snode"] != current_node else row["enode"]
            if node == current_node:
                continue
            assert node != current_node
            
            if current_type == "positive": # current node is from positive_set
                path_related(node, current_hop)
            elif current_type == "negative": # current node is from negative_set
                path_related(node, current_hop)
            elif current_type == "related":
                path_related(node, current_hop)
            # elif current_type == "controversial":
            #     path_controversial(node)
            else:
                raise Exception("Type error")

            if node not in hop_dict or hop_dict[node] > current_hop:
                hop_dict[node] = current_hop
        
    
    assert len(positive_node_dict["pending"]) == 0
    assert len(negative_node_dict["pending"]) == 0
    # assert len(neutral_node_dict["pending"]) == 0
    assert len(related_node_dict["pending"]) == 0
    assert len(controversial_node_dict["pending"]) == 0
    
    # positive_node_set = set(positive_node_dict["complete"].keys())
    # negative_node_set = set(negative_node_dict["complete"].keys())
    # neutral_node_set = neutral_node_dict["complete"]
    # related_node_set = set(related_node_dict["complete"].keys())
    # controversial_node_set = set(controversial_node_dict["complete"].keys())
    
    # return positive_node_set, negative_node_set, related_node_set, controversial_node_set
    return positive_node_dict["complete"], negative_node_dict["complete"], related_node_dict["complete"], controversial_node_dict["complete"]

In [332]:
import dataloader
class_dict = dataloader.load_class_dict(
    class_file=config.news20_class_label_path,
    class_code_column="ClassCode",
    class_name_column="ConceptNet"
)
class_labels = [class_dict[class_id] for class_id in class_dict]
print(class_labels)

['atheism', 'graphics', 'operating', 'ibm', 'mac', 'windows', 'sale', 'auto', 'motorcycle', 'baseball', 'hockey', 'crypt', 'electronics', 'medical', 'space', 'christian', 'gun', 'mideast', 'politics', 'religion']


In [323]:
# text_df = pd.read_csv(config.news20_train_path)
# sample_text = preprocess(text_df["selected_tfidf"].loc[0])

positive_set = dict()
negative_set = dict()
related_set = dict()
controversial_set = dict()

['atheism', 'graphics', 'operating', 'ibm', 'mac', 'windows', 'sale', 'auto', 'motorcycle', 'baseball', 'hockey', 'crypt', 'electronics', 'medical', 'space', 'christian', 'gun', 'mideast', 'politics', 'religion']


In [333]:
# class_labels = ["atheism", "religion", "christian"]
for cl in class_labels:
    positive_set[cl], negative_set[cl], related_set[cl], controversial_set[cl] = \
        BFS([cl], processed_conceptnet_df, max_hop=3)

BFS  atheism


1000 0 3009 68 3564


2000 0 3013 81 4205


3000 0 2797 88 4463


4000 0 2898 83 4217


...

11000 0 0 0 3620


12000 0 0 0 2651


13000 0 0 0 1688


14000 0 0 0 714


End


In [337]:
def merge_all_set(posset, negset, relset, contset):
    allin_set = dict()
    assert len(posset) == len(negset)
    assert len(posset) == len(relset)
    assert len(posset) == len(contset)
    for key in posset:
        allin_set[key] = {
            "positive": posset[key].copy(),
            "negative": negset[key].copy(),
            "related": relset[key].copy(),
            "controversial": contset[key].copy()
        }
    return allin_set

origin_all_set = merge_all_set(positive_set, negative_set, related_set, controversial_set)

In [357]:
wordlist = ["science", "physics", "physicist", "atheism",
            "physic", "energy", "lord", "darwin", 
            "evolve", "faith", "religion", "christian",
            "television", "video", "art", "graphics", 
            "graphic", "sex", "marriage", "cubic", "curve", "curves"]

for class_label in ["atheism", "graphics"]:
    print("==== %s" % class_label)
    for word in wordlist:
        find_a_word(
            word, 
            origin_all_set[class_label]
            # excluded_all_set[class_label]
        )

==== atheism
science	positive	1
physics	positive	3
physicist	positive	3
atheism	positive	0
physic	notfound
energy	positive	3
lord	positive	2
darwin	positive	3
evolve	positive	3
faith	positive	3
religion	controversial	2
christian	controversial	3
television	controversial	3
video	positive	3
art	negative	2
graphics	positive	3
graphic	related	3
sex	positive	3
marriage	positive	3
cubic	positive	3
curve	controversial	3
curves	notfound
==== graphics
science	controversial	3
physics	controversial	3
physicist	positive	3
atheism	positive	3
physic	notfound
energy	positive	3
lord	controversial	3
darwin	related	3
evolve	positive	3
faith	positive	3
religion	controversial	3
christian	positive	2
television	controversial	3
video	positive	3
art	positive	3
graphics	positive	0
graphic	positive	2
sex	controversial	3
marriage	positive	2
cubic	controversial	3
curve	controversial	3
curves	related	3


In [345]:
def find_a_word(word, pos_set, neg_set, rel_set, cont_set):
    if word in pos_set:
        print("%s\tpositive" % word)
    elif word in neg_set:
        print("%s\tnegative" % word)
    elif word in rel_set:
        print("%s\trelated" % word)
    elif word in cont_set:
        print("%s\tcontroversial" % word)
    else:
        print("%s\tnotfound" % word)

def classify_a_word(word, pos_set, neg_set, rel_set, cont_set):
    if word in pos_set:
        return "positive"
    elif word in neg_set:
        return "negative"
    elif word in rel_set:
        return "related"
    elif word in cont_set:
        return "controversial"
    else:
        return "notfound"

def find_a_word(word, class_set):
    for relation_type in class_set:
        if word in class_set[relation_type]:
            print("%s\t%s\t%d" % (word, relation_type, class_set[relation_type][word]))
            return
    print("%s\t%s" % (word, "notfound"))
    return

def classify_a_word(word, class_set):
    for relation_type in class_set:
        if word in class_set[relation_type]:
            return relation_type
    return "notfound"

In [331]:
def print_set_size(all_set):
    for class_label in all_set:
        print(class_label, " ".join([str(len(all_set[class_label][_])) for _ in all_set[class_label]]))

print_set_size(origin_all_set)

In [361]:
def exclude_other_classes(all_set):
    excluded_set = dict()
    
    for class_label in all_set:
        print(class_label)
        excluded_set[class_label] = dict()
        
        for relation in all_set[class_label]:
            excluded_set[class_label][relation] = all_set[class_label][relation].copy()
            
            for class_label_2 in all_set:
                if class_label != class_label_2:
                    for word in list(excluded_set[class_label][relation]):
                        if word in all_set[class_label_2][relation] and \
                            all_set[class_label_2][relation][word] < excluded_set[class_label][relation][word]:
                            excluded_set[class_label][relation].pop(word)
                    # excluded_set[class_label][relation] = \
                    #     excluded_set[class_label][relation] - all_set[class_label_2][relation]
    return excluded_set
                
            
    
excluded_all_set = exclude_other_classes(origin_all_set)

crypt
hockey
sale
motorcycle
operating
space
mideast
graphics


mac
electronics
gun
medical
baseball


religion
politics
windows
atheism
christian


ibm
auto


In [362]:
print_set_size(excluded_all_set)
print("=====")
print_set_size(origin_all_set)

crypt 3021 41 100 1107
hockey 2579 98 562 3100
sale 2684 133 1942 2681
motorcycle 2251 74 494 1802
space 3060 213 1066 5444
mideast 5644 0 0 1
graphics 2588 114 927 2436
mac 2815 103 459 3224
electronics 2665 163 768 3157
gun 2929 175 681 4441
medical 2388 173 1008 2650
ibm 2332 48 183 1365
baseball 3020 223 1020 4166
religion 2545 244 1819 3224
politics 2763 195 914 3769
windows 2732 105 472 2536
atheism 2708 85 405 1900
christian 2519 163 1005 3898
operating 8720 0 0 8
auto 2613 126 932 3145
=====
crypt 4259 74 167 5603
hockey 4299 146 739 8950
sale 4369 181 2163 7661
motorcycle 3790 100 632 8205
operating 10135 0 0 93
space 4498 257 1203 10114
mideast 6460 0 0 1
graphics 4344 166 1106 7568
mac 4602 164 594 8813
electronics 4291 205 948 9657
gun 4714 224 809 9633
medical 4052 217 1211 9046
baseball 4531 261 1134 9833
religion 4190 289 1996 8691
politics 4319 242 1075 9988
windows 4436 149 617 8465
atheism 4247 123 532 7264
christian 4366 219 1177 9303
ibm 3536 72 284 7329
auto 4372 1

In [227]:
# print(excluded_all_set["atheism"]["positive"])
# print(excluded_all_set["atheism"]["negative"])
class_label = "atheism"
corr = dict()
for word in excluded_all_set[class_label]["negative"]:
    if word in glove_dict:
        corr[word] = cosine(glove_dict[word], glove_dict[class_label])
# print(sorted(corr))
import operator
sorted_corr = sorted(corr.items(), key=operator.itemgetter(1))
print(sorted_corr)

[('theism', 0.4606858491897583), ('pantheism', 0.4983648657798767), ('religion', 0.523052453994751), ('orthodoxy', 0.5320606827735901), ('zoroastrianism', 0.5382743775844574), ('hinduism', 0.5487238168716431), ('islam', 0.5639428794384003), ('paganism', 0.5670253038406372), ('buddhism', 0.5719837546348572), ('mysticism', 0.5746982395648956), ('judaism', 0.5944637060165405), ('heresy', 0.6101876497268677), ('taoism', 0.6492180228233337), ('toleration', 0.6573553681373596), ('orthodox', 0.7070867419242859), ('islamic', 0.7082187533378601), ('persecution', 0.7117599248886108), ('affirmation', 0.7263973355293274), ('chastity', 0.7315566837787628), ('jehovah', 0.7407292723655701), ('antichrist', 0.7496334314346313), ('propagation', 0.754295751452446), ('falsehood', 0.7619874626398087), ('culture', 0.762374758720398), ('creed', 0.7690674960613251), ('mormon', 0.7708274126052856), ('immortality', 0.7781294137239456), ('disciple', 0.779176265001297), ('schism', 0.7822346091270447), ('meditatio

In [370]:
def check_related_words_in_document(net_all_set, class_list):
    np.set_printoptions(suppress=True, formatter={"float": lambda x: "%.3f" % x })

    for class_label in class_list:
        print("===")
        print("class from", class_label)

        # class_id_list = [1, 2, 16, 20]
        class_id_list = list(range(1, 21))
        # class_id_list = [1]
        for class_id in class_id_list:
            
            documents_for_a_class_df = text_df[text_df["class"] == class_id]

            full_stats = list()
            for idx, row in documents_for_a_class_df.iterrows():
                # if len(full_stats) > 2:
                # if len(full_stats) > 100:
                #     break
                wordlist = preprocess(row["selected_tfidf"])
                # wordlist = preprocess(row["selected"])
                # wordlist = preprocess(row["text"])
                if len(wordlist) == 0:
                    continue
                stats = np.zeros((5,))
                for w in wordlist[:50]:
                    # find_a_word(w)
                    if classify_a_word(
                            w, 
                            net_all_set[class_label]) == "positive":
                        stats[0] += 1
                    elif classify_a_word(
                            w, 
                            net_all_set[class_label]) == "negative":
                        stats[1] += 1
                    elif classify_a_word(
                            w, 
                            net_all_set[class_label]) == "related":
                        stats[2] += 1
                    elif classify_a_word(
                            w, 
                            net_all_set[class_label]) == "controversial":
                        stats[3] += 1
                    elif classify_a_word(
                            w, 
                            net_all_set[class_label]) == "notfound":
                        stats[4] += 1
                    else:
                        raise Exception("Error return from classify_a_word")
                # print("=========")
                norstats = stats / np.sum(stats)
                full_stats.append(norstats)
            full_stats = np.concatenate([full_stats], axis=0)
            # print(full_stats.shape)
            print(class_dict[class_id], np.mean(full_stats, axis=0), np.std(full_stats, axis=0))
            
def check_related_words_in_document_reverse(net_all_set, class_list):
    np.set_printoptions(suppress=True, formatter={"float": lambda x: "%.3f" % x })

    # class_id_list = [1, 2, 16, 20]
    class_id_list = list(range(1, 21))
    # class_id_list = [1]
    for class_id in class_id_list:
        print("===")
        print("document from", class_dict[class_id])

        documents_for_a_class_df = text_df[text_df["class"] == class_id]

        for class_label in class_list:
            full_stats = list()
            for idx, row in documents_for_a_class_df.iterrows():
                # if len(full_stats) > 2:
                # if len(full_stats) > 100:
                #     break
                wordlist = preprocess(row["selected_tfidf"])
                # wordlist = preprocess(row["selected"])
                # wordlist = preprocess(row["text"])
                if len(wordlist) == 0:
                    continue
                stats = np.zeros((5,))
                for w in wordlist[:50]:
                    # find_a_word(w)
                    if classify_a_word(
                            w,
                            net_all_set[class_label]) == "positive":
                        stats[0] += 1
                    elif classify_a_word(
                            w,
                            net_all_set[class_label]) == "negative":
                        stats[1] += 1
                    elif classify_a_word(
                            w,
                            net_all_set[class_label]) == "related":
                        stats[2] += 1
                    elif classify_a_word(
                            w,
                            net_all_set[class_label]) == "controversial":
                        stats[3] += 1
                    elif classify_a_word(
                            w,
                            net_all_set[class_label]) == "notfound":
                        stats[4] += 1
                    else:
                        raise Exception("Error return from classify_a_word")
                # print("=========")
                norstats = stats / np.sum(stats)
                full_stats.append(norstats)
            full_stats = np.concatenate([full_stats], axis=0)
            # print(full_stats.shape)
            print(class_label, np.mean(full_stats, axis=0), np.std(full_stats, axis=0))

===
document from atheism


atheism [0.146 0.011 0.132 0.069 0.643] [0.108 0.029 0.105 0.083 0.158]


graphics [0.137 0.007 0.095 0.135 0.626] [0.107 0.027 0.087 0.111 0.154]


operating [0.000 0.000 0.603 0.000 0.397] [0.002 0.000 0.164 0.000 0.164]


ibm [0.081 0.007 0.100 0.032 0.779] [0.078 0.020 0.086 0.054 0.121]


...

christian [0.310 0.013 0.066 0.137 0.474] [0.141 0.035 0.068 0.114 0.162]


gun [0.265 0.007 0.071 0.072 0.585] [0.141 0.025 0.068 0.081 0.155]


mideast [0.000 0.000 0.515 0.000 0.485] [0.000 0.000 0.170 0.000 0.170]


politics [0.243 0.009 0.065 0.124 0.559] [0.144 0.032 0.066 0.110 0.168]


religion [0.252 0.022 0.073 0.206 0.447] [0.152 0.046 0.073 0.127 0.161]


In [371]:
# check_related_words_in_document(excluded_all_set, class_labels)
check_related_words_in_document_reverse(excluded_all_set, class_labels)

===
document from atheism


atheism [0.146 0.011 0.132 0.069 0.643] [0.108 0.029 0.105 0.083 0.158]


graphics [0.137 0.007 0.095 0.135 0.626] [0.107 0.027 0.087 0.111 0.154]


operating [0.000 0.000 0.603 0.000 0.397] [0.002 0.000 0.164 0.000 0.164]


ibm [0.081 0.007 0.100 0.032 0.779] [0.078 0.020 0.086 0.054 0.121]


...

christian [0.310 0.013 0.066 0.137 0.474] [0.141 0.035 0.068 0.114 0.162]


gun [0.265 0.007 0.071 0.072 0.585] [0.141 0.025 0.068 0.081 0.155]


mideast [0.000 0.000 0.515 0.000 0.485] [0.000 0.000 0.170 0.000 0.170]


politics [0.243 0.009 0.065 0.124 0.559] [0.144 0.032 0.066 0.110 0.168]


religion [0.252 0.022 0.073 0.206 0.447] [0.152 0.046 0.073 0.127 0.161]


In [372]:
check_related_words_in_document(excluded_all_set, class_labels)

===
class from atheism


atheism [0.146 0.011 0.132 0.069 0.643] [0.108 0.029 0.105 0.083 0.158]


graphics [0.117 0.004 0.101 0.058 0.720] [0.121 0.021 0.117 0.087 0.172]


operating [0.083 0.002 0.089 0.054 0.772] [0.099 0.013 0.095 0.089 0.154]


ibm [0.091 0.003 0.111 0.084 0.711] [0.097 0.017 0.112 0.101 0.158]


...

christian [0.305 0.024 0.080 0.212 0.380] [0.143 0.041 0.077 0.131 0.145]


gun [0.168 0.021 0.069 0.224 0.517] [0.113 0.042 0.075 0.131 0.154]


mideast [0.186 0.021 0.097 0.162 0.535] [0.111 0.040 0.087 0.112 0.137]


politics [0.179 0.016 0.072 0.234 0.499] [0.111 0.034 0.071 0.132 0.150]


religion [0.252 0.022 0.073 0.206 0.447] [0.152 0.046 0.073 0.127 0.161]


In [272]:
tmp_df = text_df[text_df["class"] == 1]
print(tmp_df[:10]["selected_tfidf_1over10"])

0    free energy free energy by mcelwaine physicist...
1    todd faith and dogma faith and dogma are under...
2    bill wrote wrote liar lunatic originate believ...
3                           from edwin subject atheism
4    benedikt gregg jaeger not not my having my hea...
5    jon livesey objection capital punishment cruel...
6    323 lp asimov stamp due asimov stamps due asim...
7    dwyer morality what societally why follow that...
8    rushdie article gregg jaeger article article g...
9    keith ryan keith ryan require evidence if stat...
Name: selected_tfidf_1over10, dtype: object
